In [1]:
import pandas as pd

In [2]:
KEGG = [j for i in pd.read_table('../../../bionetwork/kegg.H00031.txt', header=None)[0] for j in i.split(', ') ]

In [3]:
Malacards = list(pd.read_table('../../../bionetwork/Malacards_BRCA.txt', skiprows=3, header=None)[0])

In [4]:
RegNetwork = pd.read_csv('../../../bionetwork/RegNetwork_human_exp_high.csv')

In [5]:
STRING = pd.read_table('../../../bionetwork/9606.protein.links.full.v11.5.txt', sep=' ')
gname_dict = pd.read_table('../../../bionetwork/9606.protein.info.v11.5.txt', index_col=0)['preferred_name'].to_dict()
STRING['protein1'] = STRING['protein1'].map(gname_dict)
STRING['protein2'] = STRING['protein2'].map(gname_dict)
STRING = STRING[STRING['experiments']>700]
STRING['protein12'] = STRING.apply(lambda x:''.join(sorted(set([x['protein1'],x['protein2']]))), axis=1)
STRING.drop(columns=['protein1','protein2']).duplicated().mean()

0.5

In [6]:
core_genes = sorted(set(KEGG+Malacards))
len(KEGG), len(Malacards), len(core_genes)

(112, 82, 187)

In [7]:
RegNetwork = RegNetwork.loc[RegNetwork['target_symbol'].isin(core_genes)|RegNetwork['regulator_symbol'].isin(core_genes)]['regulator_symbol	target_symbol'.split('\t')]
RegNetwork['database'] = 'RegNetwork'
RegNetwork = RegNetwork.rename(columns={'regulator_symbol':'source','target_symbol':'target'})

In [8]:
STRING = STRING.loc[STRING['protein1'].isin(core_genes)]['protein1	protein2	protein12'.split('\t')]
STRING = STRING.loc[~STRING['protein12'].duplicated()]
STRING = STRING.rename(columns={'protein1':'source','protein2':'target'}).drop(columns=['protein12'])
STRING['database'] = 'STRING'

In [9]:
network = pd.concat([RegNetwork, STRING])
network.to_csv('BRCA_related.genes.csv', index=False)

In [10]:
len(set(list(network['source']) + list(network['target'])))

871